# Setup

An example of what the model setup can do is shown here. This will be demonstrated on five metabolic models: Moraxella, H. Influenza, S. aureus, S. epidermidis and D. pigrum. 

Hence, let's import some of the methods we will use.

In [1]:
from ncmw.utils import get_models
from ncmw.setup_models import set_default_configs_and_snm3_medium, create_consistent_model, gapfill_medium, gapfill_model
models = get_models("models")
for model in models:
    print(f"Growth {model.id}: {model.slim_optimize()}")

Growth CarveMe_SNM_gapfilled_model: 0.7022727466098083
Growth himodel: 63.919604406865695
Growth Staphylococcus_epidermidis_ATCC_12228: 0.5746259500792691
Growth DP_83VPs_KB5: 0.28236539253224924
Growth iYS854: 1.3602249869877683


Notice that the growth values differ strongly. This is because the bounds and the medium differ. So lets first standardize these!

## Set default configurations

This package is devoted to the work on nasal microbioms expressed through the **SNM3** medium. Unfortunately, most metabolic models are created for the gut and thus often fail to achieve growth on SNM3. Here, 3 of the 5 models fail to achieve growth.

As a result, we cannot analyze them. However, it must be noted that when analysing a community, this may be less relevant, as other members can provide missing metabolites through interactions. On the other hand, it may be well known that this species can grow on the medium hence this indicates some problems with the metabolic model.

In [10]:
standardized_models =[]
for model in models:
    standardized_models.append(set_default_configs_and_snm3_medium(model))

In [5]:
for model in standardized_models:
    print(f"Growth {model.id}: {model.slim_optimize()}")

Growth CarveMe_SNM_gapfilled_model: 0.0
Growth himodel: 0.0
Growth Staphylococcus_epidermidis_ATCC_12228: 0.5746259500792691
Growth DP_83VPs_KB5: 0.0
Growth iYS854: 3.044356975357284


In order to allow analysis and to fix errors, we implement several "gapfilling" strategies. This adds metabolites to the medium or reactions to the model so that it can grow.

## Gapfilling

In order to achieve growth on a general predefined medium, we deploy two strategies:
* **Gapfill medium**: Since other members of the community could provide different metabolites that are currently unmodeled, the medium is less strict from the point of view of the community. Hence we may extend the medium by the minimal number of metabolites such that the organism can grow.
* **Gapfill model:** We can add reactions to the model such that it can grow on the given medium i.e. reactions such that the model can synthesize the missing metabilites.

In [11]:
for model in standardized_models:
    old_medium = model.medium
    m2, extension = gapfill_medium(model)
    print(f"We need to add following metabolites to the medium: {extension}")
    print(f"Growth {m2.id}: {m2.slim_optimize()}")
    m2.medium = old_medium # Rest for next step

We need to add following metabolites to the medium: ['EX_fe3dcit_e']
Growth CarveMe_SNM_gapfilled_model: 1.20618744198712
We need to add following metabolites to the medium: ['EX_3ump_e']
Growth himodel: 1.706848711893427
We need to add following metabolites to the medium: []
Growth Staphylococcus_epidermidis_ATCC_12228: 0.5746259500792691
We need to add following metabolites to the medium: ['EX_LalaDgluMdap_e', 'EX_ile__L_e', 'EX_metox_e']
Growth DP_83VPs_KB5: 0.28275198236758425
We need to add following metabolites to the medium: []
Growth iYS854: 3.044356975357284


Nice, this worked. It also provides use with information about missing metabolites. Note that the solution to this problem is not unique e.g. for the first result an equivalent result would be obtained by adding EX_fe3_e. 

In [13]:
for model in standardized_models:
    m2, extension = gapfill_model(model)
    print(f"We need to add following reactions to the model: {[e for e in extension]}")
    print(f"Growth {m2.id}: {m2.slim_optimize()}")

c:\users\manug\desktop\ncmw projects\ncmw\ncmw\setup_models\setup.py:66: UserWarning: The model still has no growth... . We try an alternative
  warn("The model still has no growth... . We try an alternative")
c:\users\manug\desktop\ncmw projects\ncmw\ncmw\setup_models\setup.py:81: UserWarning: The model still has no growth... . We greadily add sink reactions...
  warn("The model still has no growth... . We greadily add sink reactions...")


We need to add following reactions to the model: ['SK_fe3dcit_e']
Growth CarveMe_SNM_gapfilled_model: 1.206187441987119
We need to add following reactions to the model: [<Reaction DINSt2 at 0x1c7b7b52220>]
Growth himodel: 1.2333198465405384
We need to add following reactions to the model: []
Growth Staphylococcus_epidermidis_ATCC_12228: 0.5746259500792709
We need to add following reactions to the model: [<Reaction ACHBS at 0x1c7b1192d00>, <Reaction ASPK at 0x1c7b35fd370>, <Reaction DAPE at 0x1c7ba1746a0>, <Reaction DHAD2 at 0x1c7ba174a00>, <Reaction DHDPRy at 0x1c7ba174a30>, <Reaction DHDPS at 0x1c7ba174a60>, <Reaction HSDx at 0x1c7b2009790>, <Reaction KARA2 at 0x1c7b2009dc0>, <Reaction ASAD_1 at 0x1c7b9c7cd00>]
Growth DP_83VPs_KB5: 0.27989919666526347
We need to add following reactions to the model: []
Growth iYS854: 3.044356975357276


Nice, this worked too! Notice that the results are quit similar, but slightly different. Just adding reactions to a model is of course questionable i.e. these reaction should have some empirical evidence. One can look e.g. in KEGG if genes for certain reactions are present for a given species.

## Cleaning the model

There are typically several blocked reactions or mass charge violations within a metabolic model. Thus before checking the quality we may want to remove these e.g. using the FastCC algorithm.

In [14]:
consistent_models =[]
reports = []
for model in standardized_models:
    consistent_model, report = create_consistent_model(model)
    consistent_models.append(consistent_model)
    reports.append(report)
    

C:\Users\manug\.conda\envs\ncmw\lib\site-packages\cobra\core\group.py:107: UserWarning: need to pass in a list
  warn("need to pass in a list")
C:\Users\manug\.conda\envs\ncmw\lib\site-packages\cobra\core\group.py:107: UserWarning: need to pass in a list
  warn("need to pass in a list")


Lets look at one of the report returned.

In [5]:
reports[0]

,Blocked reactions,Metabolite formula problems,Mass charge balance violations
Original Model,246,0,122
Consistent model,0,0,117


As expected we can see some improvements.

In [15]:
for model in consistent_models:
    print(f"Growth {model.id}: {model.slim_optimize()}")

Growth CarveMe_SNM_gapfilled_model: 0.0
Growth himodel: 0.0
Growth Staphylococcus_epidermidis_ATCC_12228: 0.5746259500792683
Growth DP_83VPs_KB5: 0.0
Growth iYS854: 3.0443569753572817


And that's it. Now we may firstly check the models quality i.e. using **memote**. Next we can start analysing and comparing results for the models, or contiue to create a community model from these. This also serves as a small overview of what the "ncmw_setup" can do.